# Simulación

## Método para la selección de mapas

In [44]:
import numpy as np
import random
import pandas as pd
from itertools import combinations

def getMaps(match):
    print(match)
    map_types = {
        "control": ["Busan", "Nepal", "Ilios", "Oasis", "Lijian Tower", "Antartic Peninsula"],
        "push": ["Esperanca", "Colosseo", "New Queen Street"],
        "escort": ["Circuit Royale", "Dorado", "Route 66", "JunkerTown", "Rialto", "Havana", "Watchpoint Gibraltar", "Shambali Monastery"],
        "hybrid": ["Blizzard World", "Numbani", "Hollywood", "Eichenwalde", "Kings Row", "Midtown", "Paraiso"]
    }
    
    map_order = ["control", "hybrid", "push", "escort", "control"]
    selected_maps = []

    # Elegir el primer mapa aleatoriamente
    first_map_type = "control"
    selected_maps.append(random.choice(map_types[first_map_type]))

    # Eliminar el mapa seleccionado del tipo para el último mapa
    map_types[first_map_type].remove(selected_maps[0])
    map_order.remove(map_order[0])

    # Calcular las probabilidades para la distribución multinomial
    probabilities = [len(map_types[map_type]) for map_type in map_order]

    # Generar una selección de mapas utilizando la distribución multinomial
    selected_indices = np.random.multinomial(1, probabilities / np.sum(probabilities), size=1)[0]

    # Seleccionar un mapa aleatorio del tipo seleccionado para cada índice
    for selected_index, map_type in zip(selected_indices, map_order):
        selected_map = map_types[map_type][selected_index]
        selected_maps.append(selected_map)
        
    return selected_maps

## Generador de Encuentros

In [5]:
def getMatches():
    df = pd.read_csv('Team Comp vs Opp Comp_the big cheese_data.csv')
    df = df[df["Season"] > 2022]
    team_names = set(df["team_name"])
    # teams_matches = list(combinations(team_names,r=2))
    titans_matches = [("Vancouver Titans", i) for i in team_names if i != "Vancouver Titans"]
    
    return titans_matches

## Generador de Probabilidades

In [12]:
def getWinrateStats():
    # Read data
    dfOW = pd.read_csv('Team Comp vs Opp Comp_the big cheese_data.csv')
    dfOW.replace('Win', 1, inplace=True)
    dfOW.replace('Loss', 0, inplace=True)
    dfOW.replace('Draw', 0.5, inplace=True)
    dfOW = dfOW[~dfOW['map_name'].isin(['Hanamura', 'Paris', 'Horizon Lunar Colony', 'Temple of Anubis', 'Volskaya Industries'])]
    dfOW['Times'] = 1
    
    # Make dictionary
    teamNames = dfOW['team_name'].unique()
    mapNames = dfOW['map_name'].unique()
    mapNames = np.setdiff1d(mapNames, ['Hanamura', 'Paris', 'Horizon Lunar Colony', 'Temple of Anubis', 'Volskaya Industries'])
    
    # Get wins and times played
    dfOW.rename(columns={'opponent (the big cheese)': 'opponent_name'}, inplace=True)
    
    stats1 = dfOW.groupby(['team_name', 'opponent_name','map_name'])['Result'].sum().reset_index().rename(columns = {'0': 'frecuencia'})
    times1 = dfOW.groupby(['team_name', 'opponent_name', 'map_name'])['Times'].sum().reset_index().rename(columns = {'0': 'frecuencia'})
    
    # Get percentages
    stats = pd.merge(stats1,times1, on = ['team_name','opponent_name', 'map_name'])
    
    stats['WinRate'] = stats['Result']/stats['Times']
    
    return stats.loc[stats['team_name'] == 'Vancouver Titans']

## Simulación de Encuentros

In [ ]:
winrate_stats = getWinrateStats()
matches = getMatches()

In [45]:
# Resultados por mapa
res_maps = pd.DataFrame(columns=['Team', 'Opponent', 'Map', 'Result'])
# Resultados por match
res_matches = pd.DataFrame(columns=['Team', 'Opponent', 'Result', 'Points'])

for match in matches:
    # Mapas que se jugarán en el match actual
    maps = getMaps(match)
    # Puntos por cada mapa jugado
    maps_result = []
    
    for match_map in maps:
        # Selección del winrate para un oponente y un mapa específico
        winrate_s = winrate_stats.loc[(winrate_stats['team_name'] == 'Vancouver Titans') & (winrate_stats['opponent_name'] == match[1]) & (winrate_stats['map_name'] == match_map), 'WinRate']
        
        # En el caso de no haber jugado contra el equipo se asigna una probabilidad de victoria de 50%
        if len(winrate_s) == 0:
            winrate = 0.5
        else:
            winrate = winrate_s.values[0]
        
        # Resultados para cada match
        results = []
        # Número de iteraciones para simular cada juego
        iters = 1000
        
        # Simulación de mapa
        for i in range(iters):
            random_number = random.random()
            # Si el número aleatorio es menor al winrate el mapa se toma como ganado en la simulación (se agrega un 1 a la lista)
            results.append(1 if random_number <= winrate else 0)
        
        # Si en los resultados de la simulación el mapa se ganó más de la mitad de las veces se toma como que el mapa fue ganado
        map_result = 'Won' if sum(results) / iters > 0.5 else 'Lost'
        temp_df = pd.DataFrame([{'Team': match[0], 'Opponent': match[1], 'Map': match_map, 'Result': map_result}])
        res_maps = pd.concat([res_maps, temp_df])
        
        # Por cada mapa ganado se suma un punto y por cada mapa perdido se resta un punto
        maps_result.append(1 if map_result == 'Won' else -1)
        
        # Al ganar o perder 3 mapas se detiene la simulación y pasa al siguiente match
        if maps_result.count(1) == 3 or maps_result.count(-1) == 3:
            break
        
    temp_df2 = pd.DataFrame([{'Team': match[0], 'Opponent': match[1], 'Result': 'Won' if maps_result.count(1) == 3 else 'Lost', 'Points': sum(maps_result)}])
    res_matches = pd.concat([res_matches, temp_df2])
        
res_maps

('Vancouver Titans', 'Dreamers')
('Vancouver Titans', 'Trick Room')
('Vancouver Titans', 'Washington Justice')
('Vancouver Titans', 'Team Peps')
('Vancouver Titans', 'Florida Mayhem')
('Vancouver Titans', 'Boston Uprising')
('Vancouver Titans', 'Dallas Fuel')
('Vancouver Titans', 'PANTHERA')
('Vancouver Titans', 'Hangzhou Spark')
('Vancouver Titans', 'Vegas Eternal')
('Vancouver Titans', 'Los Angeles Valiant')
('Vancouver Titans', 'Seoul Infernal')
('Vancouver Titans', 'Atlanta Reign')
('Vancouver Titans', 'O2 Blast')
('Vancouver Titans', 'New York Excelsior')
('Vancouver Titans', 'Timeless')
('Vancouver Titans', 'Rhodes')
('Vancouver Titans', 'Poker Face')
('Vancouver Titans', 'Seoul Dynasty')
('Vancouver Titans', 'Houston Outlaws')
('Vancouver Titans', 'Guangzhou Charge')
('Vancouver Titans', 'Shanghai Dragons')
('Vancouver Titans', 'Saints')
('Vancouver Titans', 'WISP')
('Vancouver Titans', 'Redbird Esports')
('Vancouver Titans', 'San Francisco Shock')
('Vancouver Titans', 'Toronto 

,Team,Opponent,Map,Result
0,Vancouver Titans,Dreamers,Nepal,Won
0,Vancouver Titans,Dreamers,Blizzard World,Lost
0,Vancouver Titans,Dreamers,Esperanca,Won
0,Vancouver Titans,Dreamers,Circuit Royale,Won
0,Vancouver Titans,Trick Room,Busan,Lost
...,...,...,...,...
0,Vancouver Titans,Los Angeles Gladiators,Esperanca,Lost
0,Vancouver Titans,Los Angeles Gladiators,Dorado,Lost
0,Vancouver Titans,London Spitfire,Ilios,Won
0,Vancouver Titans,London Spitfire,Blizzard World,Won


In [41]:
res_matches

,Team,Opponent,Result,Points
0,Vancouver Titans,Dreamers,Lost,-1
0,Vancouver Titans,Trick Room,Won,2
0,Vancouver Titans,Washington Justice,Lost,-3
0,Vancouver Titans,Team Peps,Lost,-2
0,Vancouver Titans,Florida Mayhem,Lost,-2
0,Vancouver Titans,Boston Uprising,Won,1
0,Vancouver Titans,Dallas Fuel,Lost,-2
0,Vancouver Titans,PANTHERA,Lost,-3
0,Vancouver Titans,Hangzhou Spark,Won,1
0,Vancouver Titans,Vegas Eternal,Lost,-3


In [42]:
sum(res_matches['Points'])

-8

In [43]:
matches

[('Vancouver Titans', 'Dreamers'),
 ('Vancouver Titans', 'Trick Room'),
 ('Vancouver Titans', 'Washington Justice'),
 ('Vancouver Titans', 'Team Peps'),
 ('Vancouver Titans', 'Florida Mayhem'),
 ('Vancouver Titans', 'Boston Uprising'),
 ('Vancouver Titans', 'Dallas Fuel'),
 ('Vancouver Titans', 'PANTHERA'),
 ('Vancouver Titans', 'Hangzhou Spark'),
 ('Vancouver Titans', 'Vegas Eternal'),
 ('Vancouver Titans', 'Los Angeles Valiant'),
 ('Vancouver Titans', 'Seoul Infernal'),
 ('Vancouver Titans', 'Atlanta Reign'),
 ('Vancouver Titans', 'O2 Blast'),
 ('Vancouver Titans', 'New York Excelsior'),
 ('Vancouver Titans', 'Timeless'),
 ('Vancouver Titans', 'Rhodes'),
 ('Vancouver Titans', 'Poker Face'),
 ('Vancouver Titans', 'Seoul Dynasty'),
 ('Vancouver Titans', 'Houston Outlaws'),
 ('Vancouver Titans', 'Guangzhou Charge'),
 ('Vancouver Titans', 'Shanghai Dragons'),
 ('Vancouver Titans', 'Saints'),
 ('Vancouver Titans', 'WISP'),
 ('Vancouver Titans', 'Redbird Esports'),
 ('Vancouver Titans', 'S

In [46]:
winrate_stats

,team_name,opponent_name,map_name,Result,Times,WinRate
4275,Vancouver Titans,Atlanta Reign,Dorado,13.0,130,0.100000
4276,Vancouver Titans,Atlanta Reign,Eichenwalde,127.0,226,0.561947
4277,Vancouver Titans,Atlanta Reign,Hollywood,30.0,36,0.833333
4278,Vancouver Titans,Atlanta Reign,Ilios,18.0,72,0.250000
4279,Vancouver Titans,Atlanta Reign,King's Row,5.0,45,0.111111
...,...,...,...,...,...,...
4510,Vancouver Titans,Washington Justice,Nepal,0.0,36,0.000000
4511,Vancouver Titans,Washington Justice,New Queen Street,104.5,199,0.525126
4512,Vancouver Titans,Washington Justice,Numbani,30.0,78,0.384615
4513,Vancouver Titans,Washington Justice,Oasis,0.0,30,0.000000
